**Author**: Justine Debelius (justine.debelius@ki.se)<br>
**Date**: Summer/Fall 2021<br>
**Conda enviroment**: `micc-2021.11`<br>
**Python version**: 3.6.10<br>
**Python packages**: `pystan` (v. 2.19); `patsy` (0.5.1); <br>
**QIIME 2 version**: 2020.6<br>
**QIIME 2 plugins**: `gemeilli` (v. 0.0.7); `deicode` (v. 0.2.4)'; `empress` (v 1.1.0.dev); `songbird` (v. 1.0.4)<br>


In this notebook, we'll set up the dat to run through Stan, which does magical bayesian statistics.

**Note**: This notebook takes 24+ hours to run. Output files are saved in the data folder and can be substituted in downstream analyses if you dont want to run this step.

In [1]:
# Manages Stan and jupyter wweird convo. See:
# https://github.com/jupyter/notebook/issues/3397
import nest_asyncio
nest_asyncio.apply()

In [2]:
import os
import pickle

import biom
import numpy as np
import pandas as pd
import patsy
import stan

from qiime2 import Artifact, Metadata, Visualization

In this notebook, we'll set up the dat to run through Stan, which does magical bayesian statistics.

In [3]:
# Load the metadata 
meta_q2 = Metadata.load('data/metadata_paired.tsv')
meta = meta_q2.to_dataframe()


In [4]:
# Loads the feature table and filters the samples
table_q2 = Artifact.load('data/tables/phylum_defined_table.qza')
table = table_q2.view(pd.DataFrame)
keep_feats = (table.div(table.sum(axis=1), axis=0) > 1/1000).mean() >= 0.1
table = table.loc[meta.index, keep_feats.index[keep_feats]].copy()
# table = table

In [5]:
table.shape

(202, 243)

The data and statistics get formatted for stan, which has some strong preferences about how the data is formatted. Our first model has the tissue_num as its fixed effect, and this gets passed into the stan code.

In [6]:
meta['subject_num'] = meta['host_subject_id'].replace({
    id_: i for i, id_ in enumerate(np.sort(meta['host_subject_id'].unique()))}
).astype(int).copy() + 1
meta.sort_values(['subject_num', 'tissue_type'], inplace=True)
num_subjects = len(np.unique(meta['subject_num']))
print(num_subjects)

101


In [7]:
fixed = patsy.dmatrix('tissue_num', data=meta)
print(fixed.design_info.describe())
print(fixed.design_info.column_names)

1 + tissue_num
['Intercept', 'tissue_num']


In [8]:
code = open('scripts/stan_lme.stan', 'r').read()
print(code)

data {
  int<lower=0> N;    // number of samples
  int<lower=0> D;    // number of dimensions
  int<lower=0> J;    // number of subjects
  int<lower=0> p;    // number of covariates
  real depth[N];     // sequencing depths of microbes
  matrix[N, p] x;    // covariate matrix
  int y[N, D];       // observed microbe abundances
  int<lower=1, upper=J> subj_ids[N];   // subject ids

}

parameters {
  // parameters required for linear regression on the species means
  matrix[p, D-1] beta;                 // covariates
  matrix[J, D-1] alpha;                // subject differentials
  real<lower=0.01> disp;
}

transformed parameters {
  matrix[N, D-1] lam;
  matrix[N, D] lam_clr;
  matrix[N, D] prob;
  vector[N] z;

  z = to_vector(rep_array(0, N));
  lam = x * beta;
  // add batch effects

  // add in subject specific effects
  for (n in 1:N){
    lam[n] += alpha[subj_ids[n]];
  }
  lam_clr = append_col(z, lam);
}

model {
  // setting priors ...
  disp ~ inv_gamma(1., 1.);
  for (i in 1:D

And then we fit the data to the stan function

In [10]:
parameters =  {
    'N' : fixed.shape[0], # number of samples
    'D' : np.nan, # number of dimensions
    'J' : num_subjects, # number of subjects
    'p' : fixed.shape[1], # number of covariates
    'depth' : np.log(table.sum(axis=1).values), # depth of the sampless
    'x' : np.asarray(fixed).astype(int), # the covariance matrix - predictors 
    'y' : sub_table.loc[meta.index].values.astype(np.int64),
    'subj_ids': meta['subject_num'].values,
    }

In [11]:
print('N: \t', parameters['N'])
print('D: \t', parameters['D'])
print('J: \t', parameters['J'])
print('p: \t', parameters['p'])
print('depth: \t', parameters['depth'].shape)
print('fixed: \t', parameters['x'].shape)
print('y: \t', parameters['y'].shape)
print('subj_ids', parameters['subj_ids'].shape)


N: 	 202
D: 	 nan
J: 	 101
p: 	 2
depth: 	 (202,)
fixed: 	 (202, 2)
y: 	 (202, 3)
subj_ids (202,)


And then because Stan is run on a weird ALR system, we need to convert back to CLR for consistency and ranking...

In [1]:
def alr2clr(x):
    d = x.shape[0]
    x_clr = np.hstack((np.zeros((d, 1)), x))
    x_clr = x_clr - x_clr.mean(axis=1).reshape(-1, 1)
    return x_clr


In [13]:
def fit_stan_model(parameters, sub_table, feature_labels, design_info, 
                   sub_label='', code=code, dir_='.'):
    """
    A wrapper function to allow iterations through the world of stan
    """
    # Adds the subtable to the parameters
    parameters['D'] = sub_table.shape[1]
    parameters['y'] = sub_table.values.astype(np.int64)
    
    # Fits the model
    posterior = stan.build(code, data=parameters)
    fit = posterior.sample(num_chains=4, num_samples=1000)
    
    # Starts putting together a summary
    summary = {'fit': fit,
               'feature_labels': feature_labels,
               'model': design_info.describe(),
               'fit_columns': design_info.column_names,
               }
    
    # Performs CLR to ALR conversion ont he data
    alrs = {
        col: pd.DataFrame(alr2clr(fit['beta'][i, :, :].T).T, 
                          index=feature_labels)
        for i, col in enumerate(design_info.column_names)
    }
    
    summary['model_fits'] = alrs
    
    with open(os.path.join(dir_, f'stan_summary{sub_label}.pickle'), 'wb') as f_:
        pickle.dump(summary, f_)
    
    return summary

And now, we loop through the table of columnns, fit the data, and save it. This is partially so it takes a reasonable amount of time because this takes a *long* time to run.

In [ ]:
partial_dir = 'data/differential_ranking/tissue_fit/partial_fits'
os.makedirs(partial_dir, exist_ok=True)
stan_tissue_fits = []
for i in np.arange(0, len(table.columns), 50):
    columns = table.columns[i:(i+50)]
    fit_stan_model(parameters=parameters,
                   sub_table=table.loc[meta.index, columns], 
                   feature_labels=columns,
                   design_info=fixed.design_info,
                   sub_label=f'_fit_{i}-{i+50}',
                   dir_=partial_dir,
                   )

Building: found in cache, done.
Messages from stanc:
Warning in '/var/folders/bw/q064ds0d2795_6mxnrssf0l1gkw0rj/T/httpstan_1xvh8_4b/model_yzslwth4.stan', line 6, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/var/folders/bw/q064ds0d2795_6mxnrssf0l1gkw0rj/T/httpstan_1xvh8_4b/model_yzslwth4.stan', line 8, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/var/folders/bw/q064ds0d2795_6mxnrssf0l1gkw0rj/T/httpstan_1xvh8_4b/model_yzslwth4.stan', line 9, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will b

In [ ]:
meta['short_survival'] = (meta['long_survival'] == '0') * 1
meta['short_survival'].value_counts()

In [ ]:
fixed2 = patsy.dmatrix('tissue_num*short_survival', data=meta)
print(fixed2.design_info.describe())
print(fixed2.design_info.column_names)

In [ ]:
parameters2 =  {
    'N' : fixed2.shape[0], # number of samples
    'D' : table.shape[1], # number of dimensions
    'J' : num_subjects, # number of subjects
    'p' : fixed2.shape[1], # number of covariates
    'depth' : np.log(table.sum(axis=1).values), # depth of the sampless
    'x' : np.asarray(fixed2).astype(int), # the covariance matrix - predictors 
    'y' : table.loc[meta.index].values.astype(np.int64),
    'subj_ids': meta['subject_num'].values,
    }

In [ ]:
partial_dir = 'data/differential_ranking/interaction_fit/partial_fits'
os.makedirs(partial_dir, exist_ok=True)
for i in np.arange(200, len(table.columns), 50):
    columns = table.columns[i:(i+50)]
    fit_stan_model(parameters=parameters2,
                   sub_table=table.loc[meta.index, columns], 
                   feature_labels=columns,
                   design_info=fixed2.design_info,
                   sub_label=f'_fit_{i}-{i+50}',
                   dir_=partial_dir,
                   )

# Collating tissue data

In [ ]:
def alr2clr(x):
    d = x.shape[0]
    x_clr = np.hstack((np.zeros((d, 1)), x))
    x_clr = x_clr - x_clr.mean(axis=1).reshape(-1, 1)
    return x_clr

def extract_smmary(fp):
    with open(fp, 'rb') as f_:
        summary = pickle.load(f_)

    alrs = summary.get('model_fit', None)
    
    if alrs is not None:
        return alrs
    
    alrs = {
        col: pd.DataFrame(alr2clr(summary['fit']['beta'][i, :, :].T).T, 
                          index=summary['feature_labels'])
        for i, col in enumerate(summary['fit_columns'])
    }
    
    return alrs

In [ ]:
tissue_dir = 'data/differential_ranking/tissue_fit/partial_fits/'
tissue_pickles = os.listdir(tissue_dir)

In [ ]:
tissue_summaries = [
    extract_smmary(os.path.join(tissue_dir, cornichon))
    for cornichon in tissue_pickles
]

In [ ]:
tissue_fit = pd.concat(axis=0, objs=[
    summary['tissue_num'] for summary in tissue_summaries
])
tissue_rank = pd.DataFrame({
    'tissue_mean': tissue_fit.mean(axis=1),
    'tissue_std': tissue_fit.std(axis=1),
    'tissue_rank': np.ones(len(tissue_fit)),
})
# tissue_rank = pd.concat(axis=1, objs=[tissue_rank, taxa.loc[tissue_rank.index]])
tissue_rank.sort_values('tissue_mean', ascending=True, inplace=True)
tissue_rank['tissue_rank'] = tissue_rank['tissue_rank'].cumsum() - 1
tissue_rank.index.set_names('feature-id', inplace=True)

In [ ]:
tissue_rank.to_csv('data/differential_ranking/tissue_num.tsv', sep='\t')

# Loading interaction data

In [ ]:
inter_dir = 'data/differential_ranking/interaction_fit/partial_fits/'
inter_pickles = os.listdir(inter_dir)

inter_summaries = [
    extract_smmary(os.path.join(inter_dir, cornichon))
    for cornichon in inter_pickles
]

In [ ]:
inter_fits = {
    param: pd.concat(
        axis=0,
        objs=[
            summary[param] for summary in inter_summaries
        ]
    )
    for param in ['tissue_num', 'short_survival', 
                  'tissue_num:short_survival']
}

In [ ]:
inter_rank = pd.DataFrame({
    'tissue_mean': inter_fits['tissue_num'].mean(axis=1),
    'tissue_std': inter_fits['tissue_num'].std(axis=1),
    'tissue_rank': np.ones(len(inter_fits['tissue_num'])),
    'survival_mean': inter_fits['short_survival'].mean(axis=1),
    'survival_std': inter_fits['short_survival'].std(axis=1),
    'survival_rank': np.ones(len(inter_fits['short_survival'])),
    'inter_mean': inter_fits['tissue_num:short_survival'].mean(axis=1),
    'inter_std': inter_fits['tissue_num:short_survival'].std(axis=1),
    'inter_rank': np.ones(len(inter_fits['tissue_num:short_survival'])),
})
inter_rank.sort_values('tissue_mean', ascending=True, inplace=True)
inter_rank['tissue_rank'] = inter_rank['tissue_rank'].cumsum() - 1

inter_rank.sort_values('survival_mean', ascending=True, inplace=True)
inter_rank['survival_rank'] = inter_rank['survival_rank'].cumsum() - 1

inter_rank.sort_values('inter_mean', ascending=True, inplace=True)
inter_rank['inter_rank'] = inter_rank['inter_rank'].cumsum() - 1

inter_rank.index.set_names('feature-id', inplace=True)

In [ ]:
inter_rank.to_csv('data/differential_ranking/interaction_model.tsv', 
                  sep='\t')